---
title: "Online Feature Serving"
date: 2021-04-25
type: technical_note
draft: false
---

## Connect to the Feature Store

In [1]:
import hsfs

connection = hsfs.connection()
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
25,application_1624276539905_0005,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

## Get training dataset

In [20]:
td_meta = fs.get_training_dataset("card_fraud_model3", 1)

In [28]:
td_meta.show(10)

++
||
++
||
||
||
||
||
||
||
||
||
||
++
only showing top 10 rows

To identify which primary key names are used for this training dataset query use serving_keys method

In [29]:
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'cc_num'}

In [30]:
td_meta.show(1)

++
||
++
||
++
only showing top 1 row

For demo purposes lets prepare list of primary key values that we are interested in to buils feature vectore from online feature store

In [26]:
card_numbers = ["4867010117638802",
                 "4564139086560436", 
                 "4638396144844325",
                 "4460285888258185", 
                 "4032763187099525", 
                 "4650661577010550",
                 "4925013053127624"
                ]

In [31]:
for card_number in card_numbers:
    serving_vector = td_meta.get_serving_vector({'cc_num': card_number})
    print (serving_vector)

[]
[]
[]
[]
[]
[]
[]